# Magyar NLP - előfeldolgozás I.

A szöveges adatokban nyers formájukban még rengeteg elemzéshez irreleváns elem, zaj lehet, melyektől meg kell tisztítani a dokumentumokat.
<br>
Vannak olyan lépések, amelyeket el kell végeznünk mielőtt nekiállnánk elemezni a dokumentumgyűjteményünket.
<br>
Ebben a notebook-ban sorrol sorra átvesszük, hogy milyen előfeldolgozáson kell alávetni a dokomentumgyűjteményt, hogy elemezhető legyen.
<br>
**Tartalom:**
* Csomagok importálása,
* Fájlok beolvasása,
* Adatfeltárás,
* Korpusz tisztítása,
* Szövegbányászati elemzésekre való előkészítés
* Tisztított adatok tárolása

### ---------------------------------------------------------------------------------------


### Csomagok betöltése

In [2]:
# adatkezelés
import pandas as pd
import numpy as np

# adatelőkészítés:
import unicodedata
import re 
from datetime import datetime



### ---------------------------------------------------------------------------------------


## Fájlok beolvasása

**Stopszólista beolvasása**
<br>
A magyar stopszólistát kettő dokumentumból állítottam elő:
<br>
1. NLTK csomag: https://github.com/xiamx/node-nltk-stopwords/blob/master/data/stopwords/hungarian
2. Github: https://github.com/stopwords-iso/stopwords-hu/blob/master/stopwords-hu.txt
<br>
Ezeket összefűztem, majd "unique"-oltam (lsd. stopword_list_creator.ipybn)

In [3]:
with open("nlp_utils/stop_words.txt", "r", encoding='utf-8') as f:
    stop = [i for line in f for i in line.split('\n')]
    stop = list(filter(None, stop))

In [4]:
stop[1:10]

['ahogy',
 'ahol',
 'aki',
 'akik',
 'akkor',
 'alatt',
 'által',
 'általában',
 'amely']

**Dokumentumgyűjtemény beolvasása**

In [5]:
df_index = pd.read_csv("data/2020.csv", sep="%%", encoding="utf-8", header=0)

print("2020 index corpus loaded")
print("Length of the corpus: {} articles".format(len(df_index)))

C:\Users\molna\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


2020 index corpus loaded
Length of the corpus: 8803 articles


**Random mintavétel a korpuszból (gyorsaság miatt)**

In [6]:
df_index = df_index.sample(n=4000, replace=False, random_state=1)
print("Length of the sampled corpus: {} articles".format(len(df_index)))

Length of the sampled corpus: 4000 articles


### ---------------------------------------------------------------------------------------


## Dokumentumgyűjtemény feltárása

A dokumentumgyűjteményt érdemes pandas DataFrame objektumként tárolni, mivel az egyes cellákba beleférnek nagyon hosszú sztringek is. Valamint amikor tokenizálunk akkor a dokumentum "szózsákját" csináljuk meg, ami nem más, mint a szövegben található szavak vektora, amit egy python listában tárolhatunk. A dataframe celláiba pedig listák és szótárak is beilleszthetőek.

**Korpusz első N db sorának kiírása**

In [7]:
df_index.head(3)

,"""cim""","""datum""","""szerzo""","""tag""","""head""","""szoveg"""
"""4081""","""Hadházy összekalapozott kétmillió forintot a ...","""2020.02.20. 20:50 ""","""NA""","""Belföld,hadházy,ákos,büntetés,bírság,parlamen...","""NA""",""" Hadházy Ákos független országgyűlési képvise..."
"""7384""","""Alaposan megfogyatkoztak a turisták Budapesten""","""2020.03.14. 17:12""","""NA""","""Belföld,kínai,koronavírus,koronavírus,magyaro...","""NA""",""" Az új koronavírus okozt járvány pandémiává t..."
"""3081""","""Nézzük meg, mi szerepel a NAT-ban, és mi a ba...","""2020.02.13. 13:29""","""NA""","""Belföld,nemzeti,alaptanterv,nat,irodalom,tört...","""Ha elolvassuk a Nemzeti alaptanterv szövegét,...",""" Nagyon hamar politikai kérdéssé vált a Nemze..."


**Korpusz oszlopneveinek lekérése**

In [8]:
df_index.columns

Index(['"cim"', '"datum"', '"szerzo"', '"tag"', '"head"', '"szoveg"'], dtype='object')

In [9]:
df_index.columns = df_index.columns.str.replace('"', '')

In [10]:
df_index.columns

Index(['cim', 'datum', 'szerzo', 'tag', 'head', 'szoveg'], dtype='object')

**DataFrame info() lekérés**
<br> 
Megnézhetjük, hogy az egyes oszlopok milyen típusúak.
<br>
Object = szöveges (sztring)

In [11]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, "4081" to "3515"
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cim     4000 non-null   object
 1   datum   4000 non-null   object
 2   szerzo  4000 non-null   object
 3   tag     4000 non-null   object
 4   head    4000 non-null   object
 5   szoveg  4000 non-null   object
dtypes: object(6)
memory usage: 218.8+ KB


**További infók lekérése df.describe() függvénnyel**
<br>
Mivel még csak objekt típusú mezőink vannak a mean, medián és a percentilis információk nem elérhetőek, de érdemes megnézni a *count* és az *unique* sorokat. 
<br>
Látszik, hogy duplikációink vannak a dokumentumgyűjteményben. (Ezek kezelésére még kitérek)

In [12]:
df_index.describe()

,cim,datum,szerzo,tag,head,szoveg
count,4000,4000,4000,4000,4000,4000
unique,3868,3810,41,3807,290,3872
top,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
freq,123,97,3679,172,3708,117


### ---------------------------------------------------------------------------------------

## -1. Adattisztítási lépés

**Adatbeolvasásnál bent maradt egy nyitó és záró " - jel**

In [13]:
df_index['cim'] = df_index['cim'].apply(lambda x: x[1:-1])

In [14]:
df_index['szoveg'] = df_index['szoveg'].apply(lambda x: x[1:-1])

In [15]:
df_index['head'] = df_index['head'].str.replace('"', '')

In [16]:
df_index.head(2)

,cim,datum,szerzo,tag,head,szoveg
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,"""2020.02.20. 20:50 ""","""NA""","""Belföld,hadházy,ákos,büntetés,bírság,parlamen...",NA,Hadházy Ákos független országgyűlési képvisel...
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,"""2020.03.14. 17:12""","""NA""","""Belföld,kínai,koronavírus,koronavírus,magyaro...",NA,Az új koronavírus okozt járvány pandémiává te...


### ---------------------------------------------------------------------------------------


## Hiányzó értékek szűrése
Fontos lépés, mert ha hiányzó értékeink vannak, akkor sok függvény nem működik (.pl: split()), amelyek szükségesek a további képésekben. 
Az "NA" értékeket pedig azért is cseréltük le np.nan-ra, mert így már tudjuk őket törölni/cserélni.

**A pontosabb információkért cseréljük ki az "NA" értékeket np.nan-ra**

In [17]:
cols = ['cim', 'datum', 'szerzo', 'tag', 'head', 'szoveg']
df_index = df_index[cols].replace('NA', np.nan)

In [18]:
df_index.isna().sum()

cim        123
datum        0
szerzo       0
tag          0
head      3708
szoveg     117
dtype: int64

**Cím és szöveg mezők közül Nan-t tartalmazó sorok eldobása**

In [19]:
df_index = df_index.dropna(subset=['cim', 'szoveg'])
df_index.isna().sum()

cim          0
datum        0
szerzo       0
tag          0
head      3567
szoveg       0
dtype: int64

### ---------------------------------------------------------------------------------------

## Oszlopok összefűzése
Ez a lépés akkor hasznos, ha külön oszlopban tároljuk a címeket a szövegtörzstől, de együtt akarjuk kezelni őket. A példa adathalmazban külön tároljuk a headert (np.nan ha nincs) és a szövegtörzset, de logikailag ezek összetartoznak.

In [20]:
df_index['merged'] = df_index[['head', 'szoveg']].apply(lambda x: ','.join(x.dropna()), axis=1)

In [21]:
df_index[df_index['head'].isna()][['head', 'szoveg', 'merged']].head(2)

,head,szoveg,merged
"""4081""",NaN,Hadházy Ákos független országgyűlési képvisel...,Hadházy Ákos független országgyűlési képvisel...
"""7384""",NaN,Az új koronavírus okozt járvány pandémiává te...,Az új koronavírus okozt járvány pandémiává te...


In [22]:
df_index[~df_index['head'].isna()][['head', 'szoveg', 'merged']].head(2)

,head,szoveg,merged
"""3081""","Ha elolvassuk a Nemzeti alaptanterv szövegét, ...",Nagyon hamar politikai kérdéssé vált a Nemzet...,"Ha elolvassuk a Nemzeti alaptanterv szövegét, ..."
"""5378""","Kitalált és valódi pénzrendszerek, amelyeknek ...",A Harry Potter mesék mágikus pénznemeit így h...,"Kitalált és valódi pénzrendszerek, amelyeknek ..."


### ---------------------------------------------------------------------------------------


## Duplikációk törlése
Bármilyen oknál fogva kerülhetnek be duplikációk a dokumentumgyűjteménybe, de ezeket el kell távolítani, mivel a későbbi elemzéseket torzíthatja.

In [23]:
df_index = df_index[~df_index.duplicated(['merged'], keep = 'last')]

print("Length of the corpus without duplications: {}".format(len(df_index)))

Length of the corpus without duplications: 3851


### ---------------------------------------------------------------------------------------


## Leíró statisztikák készítése
(~miből mennyi van a szövegben)
<br>
Mielőtt nekifognánk a dokumentumokból kiszűrni az irreleváns elemeket (stopszavak, írásjelek, stb.) érdemes összeszámolni ezeket és új oszlopot létrehozni belőlük.
<br>
Ezek is fontos információk lehetnek, ha például kattintásvadász vagy álhíreket akarunk detektálni.

**Szavak száma a szövegben/címben stopszavakkal**

In [24]:
df_index['cim_word_cnt'] = df_index.cim.apply(lambda x: len(str(x).split(" "))) 
df_index[['cim', 'cim_word_cnt']][0:2]

,cim,cim_word_cnt
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,6
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,5


**Átlagos szóhoszz a címben**

In [25]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df_index['cim_avg_word'] = df_index['cim'].apply(lambda x: avg_word(x))
df_index[['cim', 'cim_avg_word']][0:2]

,cim,cim_avg_word
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,8.5
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,8.4


**Felkiáltójelek megszámolása a szövegben**

In [26]:
df_index['szoveg_num_exclam'] = df_index['szoveg'].str.count('\\!')

**Kérdőjelek megszámolása a szövegben**

In [27]:
df_index['szoveg_num_ques'] = df_index['szoveg'].str.count('\\?')

**Stopszavak száma a szövegben/címben**

In [28]:
df_index['cim_stop_cnt'] = df_index['cim'].apply(lambda x: len([x for x in x.split() if x in stop]))
df_index[['cim', 'cim_stop_cnt']].head(3)

,cim,cim_stop_cnt
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,1
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,1
"""3081""","Nézzük meg, mi szerepel a NAT-ban, és mi a baj...",5


**Nagybetűs szavak megszámolása a szövegben**

In [29]:
df_index['cim_cnt_upper'] = df_index.cim.apply(lambda x: len([x for x in x.split() if x.isupper()]))
df_index[['cim','cim_cnt_upper']][0:2]

,cim,cim_cnt_upper
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,0
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,0


### ---------------------------------------------------------------------------------------


### Szöveg (már "merged") mező tisztítása

**1. Esetenként bent maradhatnak a szövegben olyan elemek, amelyek a webscrape során bentragadtak. Megosztás, ajánló, további cikkek stb.**

Az str.contains() függvénnyel tudunk szavakra, részletekre keresni a szövegben.  

In [30]:
print(len(df_index[df_index['merged'].str.contains('\xa0')])) # 6834
print(len(df_index[df_index['merged'].str.contains('Ne maradjon le semmiről! Facebook')])) #217

3047
122


Az str.replace() függvénnyel tudunk szavakat, részleteket kiszedni, és más karakterrel helyettesíteni.

In [31]:
df_index['merged'] = df_index['merged'].str.replace('\xa0', ' ')
df_index['merged'] = df_index['merged'].str.replace('Ne maradjon le semmiről! Facebook', '')

In [32]:
print(len(df_index[df_index['merged'].str.contains('\xa0')])) # 6834
print(len(df_index[df_index['merged'].str.contains('Ne maradjon le semmiről! Facebook')])) #217

0
0


**További felesleges elemek eltávolítása**
<br>
Az adatok tisztítása iteratív folyamat. Sokszor csak az adatok feltárásánál/elemzéseknél jön ki hogy valami még nem stimmel. Pl. Az 'Akták"-ban majdnem mindíg szerepel a koronavírus, így szűrésnél majd a teljes adathalmazt visszakapjuk...
<br>
Az index-nél ezek általában a cikk utáni ajánlók, akták, megosztás, támogatás kérés...

In [36]:
df_index[df_index['merged'].str.contains('Akták A koronavírus Magyarországon.*', regex = True)]

,cim,datum,szerzo,tag,head,szoveg,merged,cim_word_cnt,cim_avg_word,szoveg_num_exclam,szoveg_num_ques,cim_stop_cnt,cim_cnt_upper


In [38]:
df_index['merged'] = df_index['merged'].apply(lambda x: re.sub("Akták A koronavírus Magyarországon.*", "", x))

In [41]:
df_index[df_index['merged'].str.contains('Támogasd te is.*', regex = True)]

,cim,datum,szerzo,tag,head,szoveg,merged,cim_word_cnt,cim_avg_word,szoveg_num_exclam,szoveg_num_ques,cim_stop_cnt,cim_cnt_upper


In [40]:
df_index['merged'] = df_index['merged'].apply(lambda x: re.sub("Támogasd te is.*", "", x))

In [42]:
df_index['merged'] = df_index['merged'].apply(lambda x: re.sub("Durva influenza vagy veszélyes világjárvány.*", "", x))

In [43]:
df_index['merged'] = df_index['merged'].apply(lambda x: re.sub("Vannak, akiknek már nincsenek kérdéseik, És vannak akik az Indexet olvassák!.*", "", x))

**2. Szövegbányászati elemzésekhez való előkészítés**
<br>
* kisbetűsítés,
* írásjelek eltávolítása,
* stopszavak eltávolítása,
* numerikus elemek eltávolítása
<br>
Mindezt egy egyszerű függvénnyel.
<br>
Itt már bevethetjük a regex eszköztárat: https://docs.python.org/3/library/re.html

In [44]:
def merged_cleaning(df):
    """
    Transforming szoveg (merged) column for the NLP:
    - Remove punctuations,
    - Lowercasing,
    - Remove stopwords.
    """
    
    df['merged_cleaned'] = df['merged'].str.replace('-', ' ')
    df['merged_cleaned'] = df['merged_cleaned'].str.replace('[^\w\s]', ' ')
    df['merged_cleaned'] = df['merged_cleaned'].str.replace('\d+', ' ') 
    
    df['merged_cleaned'] = df['merged_cleaned'].str.lower()

    df['merged_cleaned'] = df['merged_cleaned'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

    return df

In [45]:
df_index = merged_cleaning(df_index)

In [46]:
df_index.head(2)

,cim,datum,szerzo,tag,head,szoveg,merged,cim_word_cnt,cim_avg_word,szoveg_num_exclam,szoveg_num_ques,cim_stop_cnt,cim_cnt_upper,merged_cleaned
"""4081""",Hadházy összekalapozott kétmillió forintot a b...,"""2020.02.20. 20:50 ""","""NA""","""Belföld,hadházy,ákos,büntetés,bírság,parlamen...",NaN,Hadházy Ákos független országgyűlési képvisel...,Hadházy Ákos független országgyűlési képvisel...,6,8.5,1,0,1,0,hadházy ákos független országgyűlési képviselő...
"""7384""",Alaposan megfogyatkoztak a turisták Budapesten,"""2020.03.14. 17:12""","""NA""","""Belföld,kínai,koronavírus,koronavírus,magyaro...",NaN,Az új koronavírus okozt járvány pandémiává te...,Az új koronavírus okozt járvány pandémiává te...,5,8.4,1,1,1,0,koronavírus okozt járvány pandémiává terebélye...


### ---------------------------------------------------------------------------------------

## Előfeldolgozott adatok tárolása

In [47]:
df_index.to_pickle('2020_corpus_preprocessed.pkl')